In [ ]:
!pip install Cython
!pip install torch
!pip install tensorflow
!pip install keras
!pip install keras.layers.recurrent
!pip install transformers
!pip install SentencePiece

import random
import csv
import os
import re
import string
from random import randint, choice

ERROR: Could not find a version that satisfies the requirement keras.layers.recurrent (from versions: none)
ERROR: No matching distribution found for keras.layers.recurrent
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.2 MB/s eta 0:00:00


In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow.keras.layers import Dense,Flatten,BatchNormalization,Dropout,LeakyReLU
import tensorflow as tf

tf.compat.v1.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
from tensorflow import keras
from keras.models import load_model
from keras import Sequential, Model

In [ ]:
import os

!pwd

os.chdir("/content/drive/My Drive/magistrska/SloTagger/new_model_train")

os.chdir("/content/")

#os.environ['CUDA_VISIBLE_DEVICES'] = '0'
#os.environ["SM_FRAMEWORK"] = "tf.keras"

#from tensorflow import keras
#import segmentation_models as sm

#!python tag.py neoznaceno_besedilo.txt output.xml --obelikspath Obeliks4J -b 0 -e 10
#!python3 train.py ./train_corpus_1/suk_tei/ssj500k-tag.xml output -n 10 -s

/content/drive/MyDrive/magistrska/SloTagger/new_model_train


In [ ]:
# coding=utf-8

import glob, os
import sys, re
import xml.etree.ElementTree as ET

mapa = sys.argv[1]
stNepravilnoOdprtih = 0
raw_text = ""
file_path = ""
i = 0
najdaljsiTrStavek = 0
katerJeNajdaljsi = ""
skupnaDolzinaStavkov = 0
stStavkov = 0
fileNameFolders = glob.glob("/content/drive/My Drive/ucenje_BERT/gigafida/tei/GF*")

cur_mapa = "aug_16_ucne_mnozice"

skip_stevec = 0
skip_file_stevec = 0
koliko_besed = 0

skip_files = True
file_counter = 0
how_many_files_to_skip = 40
max_wanted_file_counter = 250

#testne 1-2

how_many_folders_to_skip = 4
how_many_folders_to_read = 100

print("len(fileNameFolders): ", len(fileNameFolders))

for idx_folder, fileNameFolder in enumerate(fileNameFolders):
  skip_stevec += 1
  skip_file_stevec = 0
  file_counter = 0

  print("fileNameFolder: ", fileNameFolder)

  if skip_stevec < how_many_folders_to_skip:
    continue

  if skip_stevec >= how_many_folders_to_skip + how_many_folders_to_read:
    break

  fileNames = glob.glob(fileNameFolder + "/*.xml")
  for fileName in fileNames:

    if skip_files and file_counter < how_many_files_to_skip:
      file_counter += 1
      continue

    if skip_files and file_counter > max_wanted_file_counter:
      break
    file_counter += 1

    tree = ET.parse(fileName)
    root = tree.getroot()

    text = ""

    for root_text in root[1][1]:
      for child in root_text:
        for child_with_text in child:
          if child_with_text.get('name') != 'neardup' and child_with_text.get('name') != 'std_tech_n' and child_with_text.get('name') != 'std_ling_n':
            #print(child_with_text.text)
            text += child_with_text.text
        text += '\n'

    if (file_counter % 10 == 0):
        print("fileName: ", file_counter)
    #file_counter += 1

    if not ((text.count('\n') / len(text) > 0.05) or (sum(c.isdigit() for c in text) / len(text) > 0.03) or (text.count(':') / len(text) > 0.01) or (text.count('(') / len(text) > 0.01)):


        #text = text.lower()
        text = re.sub('[!@#$:;\'\"“”•«»%&ø×¹]', '', text)
        text = re.sub('[öó]', 'o', text)
        text = re.sub('[áä]', 'a', text)
        #text = re.sub('[0123456789]', '1', text)
        text = re.sub('ë', 'e', text)
        text = re.sub('í', 'i', text)
        text = re.sub('ü', 'u', text)
        text = re.sub('ć', 'c', text)


        #odstranim odvecne presledke pred locili
        text0 = ""
        c1 = ''
        c2 = ''
        for c in text:
            c1 = c
            if ((c1 == '.' or c1 == ',' or c1 == '?' or c1 == '!') and c2 != ' ') or (c1 != '.' and c1 != ',' and c1 != '?' and c1 != '!'):
                text0 += c2
            c2 = c


        #odstranim veckratne presledke iz teksta
        text1 = ""
        c1 = ''
        c2 = ''
        for c in text0:
            c1 = c
            if not (c1 == ' ' and c2 == ' '):
                text1 += c
            c2 = c

        text2 = text1
        #izlocim stavke z *, +, -
        text3 = ""
        vmesniText = ""
        bilPlusKratMinusZnak = False
        for c in text2:
            vmesniText += c
            if c == '+' or c == '-' or c == '*':
                bilPlusKratMinusZnak = True
            elif c == '.' or c == '?' or c == '!':
                if not bilPlusKratMinusZnak:
                    text3 += vmesniText
                bilPlusKratMinusZnak = False
                vmesniText = ""

        text4 = ""
        splittedText = text3.split('\n');
        for stavek in splittedText:
          if len(stavek) > 1:
            if "." in stavek:
              text4 += stavek + "\n"

        text6 = text4

        text7 = ""
        c1 = ''
        c2 = ''
        for c in text6:
            c1 = c
            if ((c2 == ',' or c2 == '.' or c2 == '!' or c2 == '?') and c1 == '1'):
                text7 += ' ' + c
            else:
                text7 += c
            c2 = c

        #odstranim se odvecne presledke
        text5 = ""
        c1 = ''
        c2 = ''
        for c in text7:
            c1 = c
            if not (c1 == ' ' and c2 == ' '):
                text5 += c
            c2 = c
        #text5 = text4

        #print("text5 je" + text5)

        if len(text5) > 10:
            raw_text += text5
            stNepravilnoOdprtih = 0

    #print(raw_text)
    file_path = "/content/drive/My Drive/magistrska/aug_16_ucne_mnozice/aug_16_ucne_mnozice" + str(idx_folder) + "_" + str(file_counter) + ".txt"
    #print("saving in file: ", file_path)
    file = open(file_path,"w")
    file.write(raw_text)
    file.close()
    raw_text = ""





#skupnaDolzinaStavkov /= stStavkov

print("povprecnaDolzinaStavkov je", skupnaDolzinaStavkov)
print("najdalji tr stavek je", najdaljsiTrStavek)
print(katerJeNajdaljsi)

len(fileNameFolders):  100
fileNameFolder:  /content/drive/My Drive/ucenje_BERT/gigafida/tei/GF68
fileNameFolder:  /content/drive/My Drive/ucenje_BERT/gigafida/tei/GF74
fileNameFolder:  /content/drive/My Drive/ucenje_BERT/gigafida/tei/GF73
fileNameFolder:  /content/drive/My Drive/ucenje_BERT/gigafida/tei/GF41
fileName:  50
fileName:  60
fileName:  70
fileName:  80
fileName:  90
fileName:  100
fileName:  110
fileName:  120
fileName:  130
fileName:  140
fileName:  150
fileName:  160
fileName:  170
fileName:  180
fileName:  190
fileName:  200
fileName:  210
fileName:  220
fileName:  230
fileName:  240
fileName:  250
fileNameFolder:  /content/drive/My Drive/ucenje_BERT/gigafida/tei/GF13
fileName:  50
fileName:  60
fileName:  70
fileName:  80
fileName:  90
fileName:  100
fileName:  110
fileName:  120
fileName:  130
fileName:  140
fileName:  150
fileName:  160
fileName:  170
fileName:  180
fileName:  190
fileName:  200
fileName:  210
fileName:  220
fileName:  230
fileName:  240
fileName:  25

OSError: ignored

In [ ]:
#os.chdir("/content/drive/My Drive/magistrska/" + cur_mapa)
data_folder = "/content/drive/My Drive/magistrska/prepared_data/masked_data/data.csv"

In [ ]:
# Read the CSV file
with open(data_folder, 'r') as input_file:
    reader = csv.reader(input_file)
    rows = list(reader)

# Count the number of rows
num_rows = len(rows)

print(f'Number of rows: {num_rows}')

Number of rows: 5918145


In [ ]:
# Read the CSV file
with open(data_folder, 'r') as input_file:
    reader = csv.reader(input_file)
    rows = list(reader)

# Split the rows into two lists
n = 5  # Number of rows to write to the first file
first_rows = rows[:num_rows-2000]
second_rows = rows[num_rows-2000:num_rows]

# Write the first rows to a CSV file
with open('train.csv', 'w') as first_file:
    writer = csv.writer(first_file)
    # Write the header row
    writer.writerow(["input", "target"])
    writer.writerows(first_rows)

# Write the second rows to a CSV file
with open('eval.csv', 'w') as second_file:
    writer = csv.writer(second_file)
    # Write the header row
    writer.writerow(["input", "target"])
    writer.writerows(second_rows)

In [ ]:
with open(data_folder, 'r') as input_file:
    reader = csv.reader(input_file)
    rows = list(reader)


# Write the second rows to a CSV file
with open('eval.csv', 'w') as second_file:
    writer = csv.writer(second_file)
    # Write the header row
    writer.writerow(["input", "target"])
    writer.writerows(rows)

In [ ]:
!pip install happytransformer
!pip install --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 87.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 26.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=065334575c47c1b4ff2a1785add856a3f66f7181b94b7617070927dcadafe657
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8

In [ ]:
# from happytransformer import TTTrainArgs

# help(TTTrainArgs)

In [ ]:
from happytransformer import HappyTextToText

# pretrained google mt5 model
#happy_tt = HappyTextToText("T5", "google/mt5-base")

#pretrained sl t5 small model
#happy_tt = HappyTextToText("T5", "t5-sl-small", load_path="/content/drive/My Drive/magistrska/pretrained_models/t5-sl-small/t5-sl-small")

#pretrained sl t5 large model
#happy_tt = HappyTextToText("T5", "cjvt/t5-sl-large", load_path="/content/drive/My Drive/magistrska/pretrained_models/t5-sl-large")



#trained google mt5 base model
#happy_tt = HappyTextToText("T5", "google/mt5-base", load_path="/content/drive/My Drive/magistrska/ucenje_feb_15_model/")

#trained sl t5 small model
happy_tt = HappyTextToText("T5", "t5-sl-small", load_path="/content/drive/My Drive/magistrska/wo_data_aug_1/")

#trained sl t5 large model
#happy_tt = HappyTextToText("T5", "cjvt/t5-sl-large", load_path="/content/drive/My Drive/magistrska/ucenje_mar_11_slo_t5_model_large_1")

#trained sl t5 on masked words
#happy_tt = HappyTextToText("T5", "t5-sl-small", load_path="/content/drive/My Drive/magistrska/trained_models/ucenje_maj_4_masked_words_1/")




In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the pre-trained T5 model and tokenizer
model_name = "/content/drive/My Drive/magistrska/pretrained_models/t5-sl-small/t5-sl-small/"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [ ]:
import pandas as pd
from torch.utils.data import DataLoader, Dataset

class CustomDataset(Dataset):
    def __init__(self, file_path, tokenizer):
        self.data = pd.read_csv(file_path)
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_text = self.data.iloc[idx]['input']
        target_text = self.data.iloc[idx]['target']

        input_tokens = self.tokenizer.encode_plus(input_text, padding='max_length', max_length=90, return_tensors='pt', truncation=True)
        target_tokens = self.tokenizer.encode_plus(target_text, padding='max_length', max_length=90, return_tensors='pt', truncation=True)

        return {
            'input_ids': input_tokens['input_ids'].squeeze(),
            'attention_mask': input_tokens['attention_mask'].squeeze(),
            'labels': target_tokens['input_ids'].squeeze(),
            'decoder_attention_mask': target_tokens['attention_mask'].squeeze()
        }

def collate_fn(batch):
    return {key: torch.stack([sample[key] for sample in batch]) for key in batch[0]}

# Load your training data from a CSV file
train_file_path = "train.csv"
train_dataset = CustomDataset(train_file_path, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=180, collate_fn=collate_fn, shuffle=True)

In [ ]:
input_tokens_poskus = tokenizer.encode_plus("Vozil sm avto .", padding='max_length', max_length=8, return_tensors='pt', truncation=True)
target_tokens_poskus = tokenizer.encode_plus("Vozil sem avto .", padding='max_length', max_length=8, return_tensors='pt', truncation=True)

In [ ]:
print('input_ids:', input_tokens_poskus['input_ids'].squeeze())
print('attention_mask', input_tokens_poskus['attention_mask'].squeeze())
print('labels', target_tokens_poskus['input_ids'].squeeze())
print('decoder_attention_mask', target_tokens_poskus['attention_mask'].squeeze())

input_ids: tensor([1518, 5053, 3310,  670, 1662,    1,    0,    0])
attention_mask tensor([1, 1, 1, 1, 1, 1, 0, 0])
labels tensor([1518, 5053,  309,  670, 1662,    1,    0,    0])
decoder_attention_mask tensor([1, 1, 1, 1, 1, 1, 0, 0])


In [ ]:
tokenizer.convert_ids_to_tokens(input_tokens_poskus['input_ids'].squeeze().tolist())

['▁Vo', 'zil', '▁sm', '▁avto', '▁.', '</s>', '<pad>', '<pad>']

In [ ]:
tokenizer.convert_ids_to_tokens(target_tokens_poskus['input_ids'].squeeze().tolist())

['▁Vo', 'zil', '▁sem', '▁avto', '▁.', '</s>', '<pad>', '<pad>']

In [ ]:
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = optim.AdamW(model.parameters(), lr=5e-5)
num_epochs = 3

counter = 0
counter_all = len(train_dataloader) * num_epochs

model.train()
for epoch in range(num_epochs):
    total_loss = 0

    for batch in train_dataloader:
        counter += 1

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        decoder_attention_mask = batch['decoder_attention_mask'].to(device)

        # print("input_ids: ", input_ids)
        # print("attention_mask: ", attention_mask)
        # print("labels: ", labels)
        # print("decoder_attention_mask: ", decoder_attention_mask)

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, decoder_attention_mask=decoder_attention_mask)

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if counter % 50 == 0:
          avg_loss = total_loss / len(train_dataloader)
          print(f"Korak {counter} od {counter_all}. Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss}")

# Save the trained model
output_dir = "/content/drive/My Drive/magistrska/wo_data_aug_1/"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Korak 50 od 98604. Epoch 1/3, Loss: 16.572664260864258
Korak 100 od 98604. Epoch 1/3, Loss: 4.422788619995117
Korak 150 od 98604. Epoch 1/3, Loss: 1.50343918800354
Korak 200 od 98604. Epoch 1/3, Loss: 1.147467851638794
Korak 250 od 98604. Epoch 1/3, Loss: 1.0395442247390747
Korak 300 od 98604. Epoch 1/3, Loss: 0.9513142108917236
Korak 350 od 98604. Epoch 1/3, Loss: 0.8581241965293884
Korak 400 od 98604. Epoch 1/3, Loss: 0.7439712882041931
Korak 450 od 98604. Epoch 1/3, Loss: 0.7413541078567505
Korak 500 od 98604. Epoch 1/3, Loss: 0.7136843204498291
Korak 550 od 98604. Epoch 1/3, Loss: 0.6692042946815491
Korak 600 od 98604. Epoch 1/3, Loss: 0.5698890686035156
Korak 650 od 98604. Epoch 1/3, Loss: 0.6008775234222412
Korak 700 od 98604. Epoch 1/3, Loss: 0.5252691507339478
Korak 750 od 98604. Epoch 1/3, Loss: 0.5427234768867493
Korak 800 od 98604. Epoch 1/3, Loss: 0.41547366976737976
Korak 850 od 98604. Epoch 1/3, Loss: 0.47595083713531494
Korak 900 od 98604. Epoch 1/3, Loss: 0.437346696853

KeyboardInterrupt: ignored

In [ ]:
# before_result = happy_tt.eval("eval.csv")

38133


In [ ]:
# print("Before loss:", before_result.loss)

In [ ]:
# from happytransformer import TTTrainArgs

# args = TTTrainArgs(batch_size=11) #11 - 40gb gpu
# happy_tt.train("train.csv", args=args)

In [ ]:
happy_tt.save("/content/drive/My Drive/magistrska/masked_model_jul_13/")

In [ ]:
before_loss = happy_tt.eval("eval.csv")

print("After loss: ", before_loss.loss)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-63f57bb73b3bce14/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3586: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


KeyboardInterrupt: ignored

In [ ]:
from happytransformer import TTSettings

beam_settings =  TTSettings(num_beams=5, min_length=1, max_length=400, top_p=10)

In [ ]:
def compare_words(word1, word2, matchPercentage=60):
    word1_len = len(word1)
    word2_len = len(word2)
    min_len = min(word1_len, word2_len)
    matching_chars = 0

    for i in range(min_len):
        if word1[i].casefold() == word2[i].casefold():
            matching_chars += 1

    match_percentage = (matching_chars / min_len) * 100
    if match_percentage >= matchPercentage:
        return True
    else:
        return False

In [ ]:
example_1 = "Avto vozil sem."
#example_1 = split_and_replace(example_1)
print(example_1)
result_1 = happy_tt.generate_text(example_1, args=beam_settings)
print(result_1.text)

Avto vozil sem.
Avto vozil sem .


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the pretrained T5 model and tokenizer
model_name = "t5-base"
model = T5ForConditionalGeneration.from_pretrained("/content/drive/My Drive/magistrska/trained_models/general_model_jun_29/")
tokenizer = T5Tokenizer.from_pretrained("/content/drive/My Drive/magistrska/trained_models/general_model_jun_29/")

# Set the input text
input_text = "Razvoj hnologije transformerjev je prspeval k razoju velikega števila izboljšav na področju obdelave naravnega jezika."

# Tokenize the input text
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# # Generate the text
# output = model.generate(input_ids, num_return_sequences=2)

# # Decode and print the generated text
# generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
# print(generated_text)

# Generate the text using beam search
beam_outputs = model.generate(
    input_ids,
    num_beams=15,
    num_return_sequences=10,
    early_stopping=True
)

# Decode and print the generated text
for i, beam_output in enumerate(beam_outputs):
    generated_text = tokenizer.decode(beam_output, skip_special_tokens=True)
    print(f"Beam {i+1}: {generated_text}")

Beam 1: Razvoj tehnologije transformerjev je pripomogel k razoju velikega števila izboljšav na področju obdelave naravnega jezika
Beam 2: Razvoj tehnologije transformerjev je prispeval k razoju velikega števila izboljšav na področju obdelave naravnega jezika
Beam 3: Razvoj tehnologije transformerjev je prispeval k nastanku velikega števila izboljšav na področju obdelave naravnega jezika.
Beam 4: Razvoj tehnologije transformerjev je pripomogel k nastanku velikega števila izboljšav na področju obdelave naravnega jezika.
Beam 5: Razvoj tehnologije transformerjev je prispeval k razvoju velikega števila izboljšav na področju obdelave naravnega jezika.
Beam 6: Razvoj tehnologije transformerjev je pripomogel k razvoju velikega števila izboljšav na področju obdelave naravnega jezika.
Beam 7: Razvoj tehnologij transformerjev je pripomogel k razoju velikega števila izboljšav na področju obdelave naravnega jezika
Beam 8: Razvoj tehnologij transformerjev je prispeval k razoju velikega števila izbo

In [ ]:
example_1 = "s e v e d a _ n i _ p o p u š č a l _ i g r a l i _ s m o _ o c l i č n o # _ p o v s e m _ s e m _ z a d o v o l j e n # _ " #"slovnica: " + split_and_replace("moj pes runo je vsel kost .")
result_1 = happy_tt.generate_text(example_1, args=beam_settings)
print(result_1.text)

s e v e d a _ n i _ p o p u š č a l _ i g r a l i _ s m o _ o c l i č n o # _ p o v s e m _ s e m _ z a d o v o l j e n # _ 


In [ ]:
example_1 = "slovnica: grem domov v mesto novo ne."
result_1 = happy_tt.generate_text(example_1, args=beam_settings)
print(result_1.text)

Slovnica Pravim domov v Novo mesto ne grem .


In [ ]:
example_1 = "na rjavo prebarvati moram letvice."
result_1 = happy_tt.generate_text(example_1, args=beam_settings)
print(result_1.text)

Na rjavo letvice moram prebarvati .


In [ ]:
example_1 = "laser okulistiki nekaj časa uspešno uporabljajo v že ( 1995 žit / 5, str. 66 ), v plastični kirurgiji in nasploh kirurgiji ( možnost izredno natančnih in točno usmerjenih rezov, operaciji pri tkivo ne krvavi in se pozneje zaceli brez brazgotin )"
result_1 = happy_tt.generate_text(example_1, args=beam_settings)
print(result_1.text)

Laser že nekaj časa uspešno uporabljajo v okulistiki ( 1995 žit / 5 str . 66 ) v plastični kirurgiji in nasploh kirurgiji ( možnost izredno natančnih in točno usmerjenih rezov , tkivo pri operaciji ne krvavi in se pozneje zaceli brez brazgotin )


In [ ]:
example_1 = "in ta je kazen, da jo zaprejo v kamnit grob živo."
result_1 = happy_tt.generate_text(example_1, args=beam_settings)
print(result_1.text)

In ta kazen je , da jo zaprejo v živo kamnit grob .


In [ ]:
limit_test_cases = True

all_sentences = 0
matched_wo_sentences = 0
non_matched_wo_sentences = 0

all_words = 0
correct_words = 0
incorrect_words = 0

all_words_in_right_wo_sentences = 0
correct_words_in_right_wo_sentences = 0
incorrect_words_in_right_wo_sentences = 0

all_wrong_right_words = 0
correct_wrong_right_words = 0
incorrect_wrong_right_words = 0

TP = 0
FP = 0
FN = 0
TN = 0

with open("eval.csv", "r") as file:
    reader = csv.reader(file)
    counter = 0

    for line in reader:
      #we skip the firest line
      if counter != 0:
        result_line = happy_tt.generate_text(line[0], args=beam_settings)

        result_line_split = result_line.text.split()
        correct_line_split = line[1].split()

        #Izlocimo besedo, ki oznacuje nalogo, ki jo model mora opraviti
        incorrect_line_split = line[0].split()

        for index in range(0, len(result_line_split)):
          if index < len(result_line_split) and index < len(correct_line_split) and index < len(incorrect_line_split):
            if correct_line_split[index] != incorrect_line_split[index]:
              all_wrong_right_words += 1

              if not compare_words(result_line_split[index], correct_line_split[index], 100):
                incorrect_wrong_right_words += 1
              else:
                correct_wrong_right_words += 1

        for index in range(0, len(result_line_split)):
            all_words += 1

            #print("correct_line_split: ", correct_line_split)
            #print("incorrect_line_split: ", incorrect_line_split)
            #print("result_line_split: ", result_line_split)

            if index < len(incorrect_line_split) and index < len(correct_line_split):
              #print("index: ", index)
              #print("correct_line_split[index]: ", correct_line_split[index])
              #print("incorrect_line_split[index]: ", incorrect_line_split[index])
              #print("result_line_split[index]: ", result_line_split[index])

              if correct_line_split[index] != incorrect_line_split[index] and compare_words(result_line_split[index], correct_line_split[index], 100):
                #print("TP")
                TP += 1
              elif correct_line_split[index] != incorrect_line_split[index] and not compare_words(result_line_split[index], correct_line_split[index], 100):
                #print("FN")
                FN += 1
              elif correct_line_split[index] == incorrect_line_split[index] and compare_words(result_line_split[index], correct_line_split[index], 100):
                #print("TN")
                TN += 1
              elif correct_line_split[index] == incorrect_line_split[index] and not compare_words(result_line_split[index], correct_line_split[index], 100):
                #print("FP")
                FP += 1
              else:
                print('NAPAKA')
              # if not compare_words(result_line_split[index], correct_line_split[index], 100):
              #   #print("padel pri: ", result_line_split[index], " in ", correct_line_split[index])
              #   incorrect_words += 1
              # else:
              #   correct_words += 1


        all_same = 1

        if len(result_line_split) == len(correct_line_split):
          for index in range(0, len(result_line_split)):
            all_words_in_right_wo_sentences += 1

            if not compare_words(result_line_split[index], correct_line_split[index], 100):
              #print("padel pri: ", result_line_split[index], " in ", correct_line_split[index])
              incorrect_words_in_right_wo_sentences += 1
            else:
              correct_words_in_right_wo_sentences += 1

            if not compare_words(result_line_split[index], correct_line_split[index]):
              all_same -= 1
              #print("padel pri: ", result_line_split[index], " in ", correct_line_split[index])

              if all_same == 0:
                break
        else:
          all_same = 0

        if all_same > 0:
          matched_wo_sentences += 1
        else:
          non_matched_wo_sentences += 1

          #print("equal length, but different words")
          #print(result_line_split)
          #print(correct_line_split)

        all_sentences += 1

      counter += 1
      if limit_test_cases and counter > 100:
        break

print("all_sentences: ", all_sentences)
print("matched_wo_sentences: ", matched_wo_sentences)
print("non_matched_wo_sentences: ", non_matched_wo_sentences)

print("all_words_in_right_wo_sentences: ", all_words_in_right_wo_sentences)
print("correct_words_in_right_wo_sentences: ", correct_words_in_right_wo_sentences)
print("incorrect_words_in_right_wo_sentences: ", incorrect_words_in_right_wo_sentences)

print("all_words: ", all_words)
print("correct_words:", correct_words)
print("incorrect_words:", incorrect_words)

print("all_wrong_right_words: ", all_wrong_right_words)
print("correct_wrong_right_words: ", correct_wrong_right_words)
print("incorrect_wrong_right_words: ", incorrect_wrong_right_words)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


all_sentences:  100
matched_wo_sentences:  61
non_matched_wo_sentences:  39
all_words_in_right_wo_sentences:  1491
correct_words_in_right_wo_sentences:  1461
incorrect_words_in_right_wo_sentences:  30
all_words:  2121
correct_words: 0
incorrect_words: 0
all_wrong_right_words:  475
correct_wrong_right_words:  323
incorrect_wrong_right_words:  152


In [ ]:
print("TP:", TP)
print("FP:", FP)
print("TN:", TN)
print("FN:", FN)

TP: 323
FP: 151
TN: 1492
FN: 152


In [ ]:
accuracy_right_wo_sentences = correct_words_in_right_wo_sentences / all_words_in_right_wo_sentences
accuracy_all_sentences = correct_words / all_words
accuracy_all_wrong_right_words = correct_wrong_right_words / all_wrong_right_words

print("accuracy_right_wo_sentences: ", accuracy_right_wo_sentences)
print("accuracy_all_sentences: ", accuracy_all_sentences)
print("accuracy_all_wrong_right_words: ", accuracy_all_wrong_right_words)

accuracy_right_wo_sentences:  0.9515449438202247
accuracy_all_sentences:  0.9373528026377768
accuracy_all_wrong_right_words:  0.9414483821263482


In [ ]:
from google.colab import runtime
runtime.unassign()

In [ ]:
def split_and_replace(input_str):
    slovenian_alphabet = set('abcčdefghijklmnoprsštuvzž')
    output_str = ''
    for char in input_str:
        if char == ' ':
            output_str += '_'
        elif char.lower() not in slovenian_alphabet:
            output_str += '#'
        else:
            output_str += char
        output_str += ' '
    return output_str

def de_split(input_str):
    return re.sub('_', ' ', input_str)

In [ ]:
!pip install datasets